# Document Analysis Assignment 5: Information Extraction

## Your Information
Please fill in the following information:

- **Name**:    [You Li]

- **Uni id**:  [u6430173]


## Overview

In this assignment, the task is to code a Named Entity Recognizer (NER) application in Python using the CRFsuite library.

To complete this task, follow the tutorial NamedEntityExtraction.ipynb and the ie-assignment-instructions.ipynb instructions posted in Wattle.

The following items summaries the assigment tasks:
1. Built a NER classifier following the tutorial.
2. Write a Python NER application that used your classifier.
3. Submit your results in Kaggle.
3. Answerd three written assignments.


* We will check the correctness of your code, but the score of the programming assignments will be graded based on your performance on Kaggle competition.
* Write your code after ### Your code here, and remove raise NotImplementedError after implementation.
* Written assignments should be written in the given notebook cells. Please write them direcly in to the designated cells, and upload the notebook file to Wattle page.
* Write answers in this notebook file, and upload the file to Wattle submission site. **Please rename and submit jupyter notebook file (Assignment5.ipynb) to your_uid.ipynb (e.g. u6000001.ipynb) with your written answers therein**. Do not upload any other files to Wattle except this notebook file.


## For the Kaggle competition

Join the competition [here](https://www.kaggle.com/t/8cedc437b8d84b8f8710ecb74e96e349).
Before submitting the result, first go to team menu and change your team name as your university id.
You need to upload the generated result file to Kaggle. 

Note that you are only allowed to upload 5 copies of your results to Kaggle per day. Make every upload count, and don't waste your opportunities!
You should use cross-validation instead of relying on the public set - this is what the daily limit is for!
**Note:** you need to fill in the cells below with your code. Failure to provide your code nullifies your Kaggle grade (meaning you get zero for the coding part).

# 1. Build a NER model (4 points) <a id='Task1'></a>

* You can use the code provided in [tutorial sheet](Tutorial/Named_Entity_Extraction_Tutorial.ipynb) 


In [5]:
from __future__ import print_function
from sklearn.metrics import confusion_matrix
import io
import nltk
import scipy
import codecs
import sklearn
import pycrfsuite
import pandas as pd
from itertools import chain
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
# import enchant
# from langdetect import DetectorFactory 

print('sklearn version:', sklearn.__version__)
print('Libraries succesfully loaded!')


sklearn version: 0.19.1
Libraries succesfully loaded!


In [21]:
# Load train and test data
train_data, train_ids = extract_data('train')
test_data, test_ids = extract_data('test')

In [34]:
def sent2features(sent, feature_func):
    return [feature_func(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    #print('sent', sent)
    return [s[-1] for s in sent]

def sent2tokens(sent):
    return [s[0] for s in sent]

def bio_classification_report(y_true, y_pred):
    """
    Classification report for a list of BIO-encoded sequences.
    It computes token-level metrics and discards "O" labels.
    
    Note that it requires scikit-learn 0.15+ (or a version from github master)
    to calculate averages properly!
    """
    lb = LabelBinarizer()
    y_true_combined = lb.fit_transform(y_true)
    y_pred_combined = lb.transform(y_pred)
        
    tagset = set(lb.classes_) - {'O'}
    tagset = sorted(tagset, key=lambda tag: tag.split('-', 1)[::-1])
    class_indices = {cls: idx for idx, cls in enumerate(lb.classes_)}
    
    return classification_report(
        y_true_combined,
        y_pred_combined,
        labels = [class_indices[cls] for cls in tagset],
        target_names = tagset,
    )
# load data and preprocess
def extract_data(path):
    """
    Extracting data from train file or test file. 
    path - the path of the file to extract
    
    return:
        res - a list of sentences, each sentence is a
              a list of tuples. For train file, each tuple
              contains token and label. For test file, each
              tuple only contains token.
        ids - a list of ids for the corresponding token. This
              is mainly for Kaggle submission.
    """
    #with open(path) as file:
    file = io.open(path, mode="r", encoding="utf-8")
    next(file)
    res = []
    ids = []
    sent = []
    for line in file:
        if line != '\n':
            parts = line.strip().split(' ')
            sent.append(tuple(parts[1:]))
            ids.append(parts[0])
        else:
            res.append(sent)
            sent = []
                
    return res, ids

# english_dict = enchant.Dict("en_US") #also available are en_GB, fr_FR, etc

# english_dict.check("Hello") # prints True
# english_dict.check("Helo") #prints False
            
def word2simple_features(sent, i):
    '''
    This makes a simple baseline.  
    You can add and/or remove features to get (much?) better results.
    Experiment with it as you will need to do this for assignment.
    '''
    word = sent[i][0]
    
    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-2:]': word[-2:],
        'word[-3:]': word[-3:],
        'word[-4:]': word[-4:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
#         'word.isEnglish()': english_dict.check(word)
    }
    if i == 0:
        features['BOS'] = True
        
    if i == len(sent)-1:
        features['EOS'] = True

    return features

            

In [35]:
print('Train and Test data upload succesfully!')

# Feature extraction using the word2simple_features function
train_features = [sent2features(s, feature_func=word2simple_features) for s in train_data]
train_labels = [sent2labels(s) for s in train_data]
test_features = [sent2features(s, feature_func=word2simple_features) for s in test_data]

# print (train_features)

trainer = pycrfsuite.Trainer(verbose=False)
for xseq, yseq in zip(train_features, train_labels):
    trainer.append(xseq, yseq)
print('Feature Extraction done!')    

# Explore the extracted features    
# sent2features(train_data[0], word2simple_features)

Train and Test data upload succesfully!
Feature Extraction done!


In [44]:
trainer.select('l2sgd')
trainer.params()
#, 'lbfgs', 'l2sgd', 'ap', 'pa', 'arow'

['feature.minfreq',
 'feature.possible_states',
 'feature.possible_transitions',
 'c2',
 'max_iterations',
 'period',
 'delta',
 'calibration.eta',
 'calibration.rate',
 'calibration.samples',
 'calibration.candidates',
 'calibration.max_trials']

In [50]:
trainer.set_params({
    'feature.minfreq': 0,
#     'feature.possible_states': 8,
#     'c1': 0.01,   # coefficient for L1 penalty
    'c2': 1e-3,  # coefficient for L2 penalty
    'max_iterations': 50,  # stop earlier
    'calibration.rate': 0.01,
    'calibration.samples': 10,


    # include transitions that are possible, but not observed
    'feature.possible_transitions': False
})

In [51]:
%%time
trainer.train('ner-esp.model')

print('Training done :)')

Training done :)
CPU times: user 13.2 s, sys: 271 ms, total: 13.4 s
Wall time: 13.7 s


In [52]:
# Make predictions
tagger = pycrfsuite.Tagger()
tagger.open('ner-esp.model')

# from sklearn.model_selection import train_test_split, KFold

# X_train, X_test, y_train, y_test = train_test_split(
#     train_features, train_labels, test_size=0.2, random_state=1)
# y_pred = [tagger.tag(xseq) for xseq in X_test]

# y_test = [item for sublist in y_test for item in sublist]
# y_pred = [item for sublist in y_pred for item in sublist]
# print(bio_classification_report(y_pred, y_test))

# import numpy as np
# X = np.array(train_features)
# y = np.array(train_labels)
# kf = KFold(n_splits=2)
# kf.get_n_splits(X)
# KFold(n_splits=2, random_state=None, shuffle=False)
# for train_index, test_index in kf.split(X):
#     print("TRAIN:", train_index, "TEST:", test_index)
#     X_train, X_test = X[train_index], X[test_index]
#     y_train, y_test = y[train_index], y[test_index]

# y_pred = [tagger.tag(xseq) for xseq in X_test]
# y_test = [item for sublist in y_test for item in sublist]
# y_pred = [item for sublist in y_pred for item in sublist]
# print(bio_classification_report(y_pred, y_test))






The output of the above cell should look something like this (ignored the numbers)

                precision    recall  f1-score   support

      B-LOC       0.68      0.47      0.55      1084
      I-LOC       0.52      0.25      0.34       325
     B-MISC       0.54      0.11      0.19       339
     I-MISC       0.54      0.22      0.32       557
      B-ORG       0.76      0.51      0.61      1400
      I-ORG       0.67      0.44      0.53      1104
      B-PER       0.73      0.68      0.71       735
      I-PER       0.78      0.82      0.80       634

avg / total       0.68      0.48      0.55      6178



## 2. Generate your result file for Kaggle

In [53]:
def generate_kaggle_res_file(ids, labels, file_path):
    """
    Generate result file for submitting to Kaggle.
    ids - the id for the tokens in test file
          should be in the same order as test file
    labels - the predictted label for each token
    file_path - the path includes the filename where
                you want to save the result
    """
    with open(file_path, 'w') as res_file:
        res_file.write('id,label\n')
        for i,l in zip(ids, labels):
            res_file.write('{},{}\n'.format(i,l))
            
test_pred = [tagger.tag(xseq) for xseq in test_features]
# print(len(test_pred)) #3232
test_pred = [s for w in test_pred for s in w]
# print(len(test_pred)) #110973 split up


# Generate Kaggle file
generate_kaggle_res_file(test_ids, test_pred, 'result.csv')

## Print evaluation
# true_labels = [item for sublist in train_labels for item in sublist]

# print(bio_classification_report(test_pred, test_labels))



In [54]:
print (len(trainer.logparser.iterations), trainer.logparser.iterations[-1])

50 {'loss': 5923.742288, 'num': 50, 'time': 0.268, 'scores': {}}


In [42]:
from collections import Counter
info = tagger.info()

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(info.transitions).most_common(15))

print("\nTop unlikely transitions:")
print_transitions(Counter(info.transitions).most_common()[-15:])

Top likely transitions:
I-PER  -> I-PER   21.371134
B-PER  -> I-PER   20.847619
B-LOC  -> I-LOC   18.044348
I-LOC  -> I-LOC   17.475985
B-ORG  -> I-ORG   15.658785
I-MISC -> I-MISC  15.579910
I-ORG  -> I-ORG   15.215636
B-MISC -> I-MISC  14.864139
O      -> B-MISC  11.301012
O      -> B-PER   11.285724
O      -> B-ORG   11.277918
O      -> O       8.282484
O      -> B-LOC   7.180390
B-LOC  -> B-LOC   6.751599
I-PER  -> B-PER   6.494097

Top unlikely transitions:
I-ORG  -> B-MISC  0.168685
B-PER  -> B-LOC   0.167065
B-PER  -> O       -0.208186
I-LOC  -> O       -0.391371
B-ORG  -> O       -1.109475
B-MISC -> O       -1.457623
B-MISC -> B-LOC   -1.493944
I-LOC  -> B-LOC   -1.832502
I-MISC -> O       -1.886312
I-ORG  -> O       -2.481942
B-LOC  -> B-ORG   -2.545534
I-MISC -> B-LOC   -2.797817
B-PER  -> B-PER   -2.992206
I-ORG  -> B-LOC   -4.157652
B-ORG  -> B-ORG   -5.102596


In [33]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-6s %s" % (weight, label, attr))    

print("Top positive:")
print_state_features(Counter(info.state_features).most_common(20))

print("\nTop negative:")
print_state_features(Counter(info.state_features).most_common()[-20:])

Top positive:
20.727089 O      word.lower():o
19.351184 B-ORG  word.lower():bnp-paribas
19.144133 I-MISC word.lower():extraviado
19.068087 O      word[-2:]:73
18.948370 B-ORG  word.lower():psoe-progresistas
18.944462 B-MISC word.lower():firagran
18.560838 I-MISC word.lower():firagran
18.545395 B-ORG  word.lower():efe-cantabria
17.190828 B-MISC word.lower():life-naturaleza
17.100157 B-ORG  word[-2:]:-e
16.875733 B-LOC  word.lower():baleares
16.397091 I-ORG  word.lower():barbara
16.362945 B-ORG  word.lower():coag-extremadura
16.211379 B-ORG  word.lower():eu-ecologista
15.786045 B-ORG  word.lower():petrobras
15.226565 B-ORG  word.lower():alcampo
15.204431 I-LOC  word.lower():rose
15.085947 B-PER  word.lower():mcmanaman
14.235392 B-ORG  word.lower():gestion
14.036794 O      word.lower():amistad

Top negative:
-5.692406 O      word.lower():sudeste
-5.752065 O      word.lower():humanos
-5.762225 O      word.lower():convento
-5.791521 O      word.lower():agricultura
-5.813972 O      word.lowe

## 3. Written part (6 pts)

Answer briefly and concisely the following questions.
Provide answers using bullet list with 2~3 items.
Check [this](https://sourceforge.net/p/jupiter/wiki/markdown_syntax/#md_ex_lists) if you are not familiar with markdown syntax.
Each questions is worth 1 mark (10%) of the total grade for the IE assignment.

### Question 1 (2 pts)
Think about three relevant baselines for the Named Entity Classification task.
Provide answers using bullet list with 3 items. Give a short description of each of them.

The baseline is defined as the information that used as a starting point bt which to compare other information.
In benchmarking, the baseline describes the simple approaches of the selected learning methods. Overall, it including random word assignment, majority class voting, heuristics, machine learning methods, and simple feature sets.

* The first baseline is knowledge based feature sets. In this specific NER Classification task, The first baseline implements a knowledge-based approach and is based on a set of dictionaries and hand-crafted rules. In word2simple_features function, word identity, word suffix, word shape and word POS tag (label) were used for train the model.

* The second baseline implements a statistical approach on features extracted. In this machine-learning system, namely a linear-chain conditional random field (CRF) with Viterbi inference that uses full corpus statistics. Cross-validation over the markup of the corpus was performed in training the data set.

* The third baseline is implementing suitable algorithms and parameters for training the model. Multiple training algorithms were built for train the learning model, including lbfgs, l2sgd, ap, pa, and arow which delivers different performance. Changing the parameters for each model also provides variety of training results.


### Question 2 (2 pts)
How the Maximal Marginal Relevance (MMR) addresses redundancy issues? (1 point)
How can you tell MMR that "Sydney" and "Melbourne" are cities? (0.5 points)
How can you tell MMR that "solar panels" and "photovoltaic cells" have similar meaning? (0.5 points)

* According to Carbonell & Goldstein, 1998, the Maximal Marginal Relevance, MMR is defines as follow:

$$
M M R \stackrel { \mathrm { def } } { = } \operatorname { arg } \max _ { D _ { i } \in R \backslash S } \left[ \lambda \left( \operatorname { Sim } _ { 1 } \left( D _ { i } , Q \right) - ( 1 - \lambda ) \max _ { D _ { j } \in S } \operatorname { sim } _ { 2 } \left( D _ { i } , D _ { j } \right) \right) \right]
$$
Where 

Given the above definition, MMR computes incrementally the standard relevance-ranked list when the parameter $\lambda=1$, and computes a maximal diversity ranking among the documents in R when $\lambda=0$. This is a linear combination of both criteria is optimized. Users wishing to sample the information space around the query, should set $\lambda$ closer to 0; in contrast, if users want focus on multiple potentially overlapping or reinforcing relevant documents, should set $\lambda$ closer to 1. In other words, the redundancy has strong relation with $\lambda$.


* In the above definition, $Sim1$ measures the relevance between an item and a query. Hence it could be done by difining ontological relations of Sydney and Mulburn as instances of city. Besides, defining words with first capitalised or Gazzeters could be useful. This word relation could be to the ranked list $R$. However this is an ambiguity prolblems of words with upper case.

* In the above definition, $Sim2$ measures the relevance between between two items. The items "solar panels" and "photovoltaic cells" have synonym relation of "solar" and "photovoltaic", and the word root as "sol-" and "photo-". "sol-" related "sun" word family:  helio-, and "photo-" Etymologically related "light, shine, glow" word families: 
 ethero-;
 helio-.
 lustr-; 
 phengo-; 
 pheno-; 
 phospho-;
 scinti-, scintill-;
 splendo-.
So the connection could be built by set realations of word roots, and send the word relation to the ranked list.



### Question 3 (2 pts)

Imagine you are developing an extractive text summarization tool using HMM.
What are the hidden states and the observations of the HMM model?
Which algorithm is use to compute the prob. of a particular observation sequence?

<img src="HMM.png">

* The HMM model is shown as above.
    * $Q = q _ { 1 } q _ { 2 } \dots q _ { N } \quad$ a set of $N$ states

    * $A = a _ { 11 } \ldots a _ { i j } \ldots a _ { N N } \quad$ a transition probability matrix $A ,$ each $a _ { i j }$ representing the probability of moving from state $i$ to state $j \\$ s.t. $\sum _ { j = 1 } ^ { N } a _ { i j } = 1 \quad \forall i$

    * $O = o _ { 1 } o _ { 2 } \ldots o _ { T } \quad$ a sequence of T observations, each one drawn from a vocabulary $V =v _ { 1 } , v _ { 2 } , \dots , v _ { V }$

    * $B = b _ { i } \left( o _ { t } \right) \quad$ a sequence of observation likelihoods, also called emission probabilities, each expressing the probability of an observation $o _ { t }$ being generated from a state $i$

    * $\pi = \pi _ { 1 } , \pi _ { 2 } , \ldots , \pi _ { N } \quad$ an initial probability distribution over states. $\pi _ { i }$ is the probability that the Markov chain will start in state i. Some states j may have $\pi _ { j } = 0$, meaning that they cannot be initial states. Also, $\sum _ { i = 1 } ^ { n } \pi _ { i } = 1$

* Hidden states is the set Q of X and observations is the sequence of O.

    * To illustrate, in the problem of icecream weather, set Q of H-H-C is hidden state and set O of 3-3-1 is the observations.

* Viterbi algorithm is used for decoding problems for HMM. 

    * This is an Dynamic programming algorithm, use trellis to store probabilities that the HMM is in state $j$ after seeing the first $t$ observations, for all states $j$. Best path Extension of a path from state $i$ at time $t-1$ is computed.
$$
v _ { t } ( j ) = \max _ { i = 1 } v _ { t - 1 } ( i ) a _ { i j } b _ { j } \left( o _ { t } \right)
$$
To illustrate, following image shows the process of Viterbi. In each state, posiboity of H/F were calculated accordign to transition parameters A and emission parameters B. Then the move is made from the node with highest value.


<img src="Viterbi.png">

